In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
from math import sqrt
from sklearn.metrics import mean_squared_error, mean_absolute_error

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from matplotlib import gridspec

In [2]:
# 字典编码函数
def change_object_cols(se):
    value = se.unique().tolist()
    value.sort()
    return se.map(pd.Series(range(len(value)),index=value)).values

In [50]:
df=pd.read_csv('jinghu.csv')
df.head()

,Unnamed: 0,date,train_number,train_direction,station_name,station_order,scheduled_arrival_time,scheduled_departure_time,stop_time,actual_arrival_time,actual_departure_time,arrival_delay,departure_delay,wind,weather,temperature,major_holiday
0,0,2019-10-08,G102,up,Shanghaihongqiao Railway Station,1,06:26:00,06:26:00,----,06:25:00,06:25:00,0,-1,gentle breeze from the NW,cloudy,21,False
1,1,2019-10-08,G102,up,Wuxidong Railway Station,2,06:54:00,06:56:00,2,06:54:00,06:55:00,0,-1,light winds from the N,cloudy,20,False
2,2,2019-10-08,G102,up,Changzhoubei Railway Station,3,07:13:00,07:15:00,2,07:12:00,07:14:00,-1,-1,gentle breeze from the E,overcast,20,False
3,3,2019-10-08,G102,up,Nanjingnan Railway Station,4,07:46:00,07:48:00,2,07:45:00,07:47:00,-1,-1,light winds from the NE,overcast,21,False
4,4,2019-10-08,G102,up,Chuzhou Railway Station,5,08:06:00,08:08:00,2,08:05:00,08:06:00,-1,-2,light winds from the NE,cloudy,21,False


风向和风拆开

eg.gentle breeze from the E  -->
 - gentle breeze
 - the E

In [51]:
df['wind']=df['wind'].apply(lambda x: x.split('from'))

In [52]:
df['wind direction']=df['wind'].apply(lambda x: x[-1])

In [53]:
df['wind']=df['wind'].apply(lambda x: x[0])
df.head()

,Unnamed: 0,date,train_number,train_direction,station_name,station_order,scheduled_arrival_time,scheduled_departure_time,stop_time,actual_arrival_time,actual_departure_time,arrival_delay,departure_delay,wind,weather,temperature,major_holiday,wind direction
0,0,2019-10-08,G102,up,Shanghaihongqiao Railway Station,1,06:26:00,06:26:00,----,06:25:00,06:25:00,0,-1,gentle breeze,cloudy,21,False,the NW
1,1,2019-10-08,G102,up,Wuxidong Railway Station,2,06:54:00,06:56:00,2,06:54:00,06:55:00,0,-1,light winds,cloudy,20,False,the N
2,2,2019-10-08,G102,up,Changzhoubei Railway Station,3,07:13:00,07:15:00,2,07:12:00,07:14:00,-1,-1,gentle breeze,overcast,20,False,the E
3,3,2019-10-08,G102,up,Nanjingnan Railway Station,4,07:46:00,07:48:00,2,07:45:00,07:47:00,-1,-1,light winds,overcast,21,False,the NE
4,4,2019-10-08,G102,up,Chuzhou Railway Station,5,08:06:00,08:08:00,2,08:05:00,08:06:00,-1,-2,light winds,cloudy,21,False,the NE


In [54]:
df['wind']=df['wind'].apply(lambda x: x.strip())

In [55]:
df['wind direction']=df['wind direction'].apply(lambda x: x.strip())

<hr>

修改风级

In [56]:
df['wind'].value_counts()

light winds        34062
gentle breeze      12298
moderate breeze     3561
fresh breeze         266
strong breeze          4
Name: wind, dtype: int64

一般的风 wind

微风 breeze

气象学上风的级别：
- 0级：Calm （无风）
- 1级：Light air（软风）
- 2级：Light breeze （轻风）
- 3级： Gentle breeze （微风）
- 4级：Moderate breeze （和风）
- 5级：Fresh breeze （清劲风）
- 6级：Strong breeze （强风）
- 7级：Moderategale (疾风)
- 8级： Fresh gale （大风）
- 9级：Strong gale （烈风）
- 10级：Whole gale（狂风）
- 11级：Storm (暴风）
- 12级： Hurricane（台风，又名飓风）
- 13级：TY （台风）
- 14、15级：STY（强台风）
- 16、17级：Super TY （超强台风）

In [57]:
def wind_level(x):
    y = 0
    if x == 'gentle breeze':
        y = 0.25
    elif x == 'moderate breeze':
        y = 0.5
    elif x == 'fresh breeze':
        y = 0.75
    elif x == 'strong breeze':
        y = 1
    return y

In [58]:
df['wind']=df['wind'].apply(wind_level)
df['wind'].value_counts()

0.00    34062
0.25    12298
0.50     3561
0.75      266
1.00        4
Name: wind, dtype: int64

<hr>

In [59]:
df['wind direction'].value_counts()

the N          11406
the NE          9241
the S           7443
the NW          5826
the E           5433
the SW          3835
the SE          3664
the W           2122
light winds     1221
Name: wind direction, dtype: int64

<hr>
天气

In [60]:
df['weather'].value_counts()

sunny            18528
cloudy           18122
light rain        6597
overcast          4772
sleet              609
moderate rain      523
light snow         371
fog                368
haze               120
heavy snow          91
moderate snow       53
showers             37
Name: weather, dtype: int64

In [61]:
def weather_num(x):
    weather = ['sunny','cloudy','overcast','sleet','light rain','light snow','haze',
               'fog','moderate rain', 'moderate snow', 'showers','heavy snow']
    if x in weather:
        return weather.index(x)+1

In [62]:
weather = df['weather'].apply(weather_num)

In [63]:
weather.value_counts()

1     18528
2     18122
5      6597
3      4772
4       609
9       523
6       371
8       368
7       120
12       91
10       53
11       37
Name: weather, dtype: int64

In [64]:
def weather_level(x):
    x = int(x)
    y = 0.0
    if x == 1:
        y = 0.0
    elif x == 12:
        y = 1.0
    else:
        y = (x - 1)/11
    return y

In [65]:
weather_level = weather.apply(weather_level)
weather_level.value_counts()

0.000000    18528
0.090909    18122
0.363636     6597
0.181818     4772
0.272727      609
0.727273      523
0.454545      371
0.636364      368
0.545455      120
1.000000       91
0.818182       53
0.909091       37
Name: weather, dtype: int64

In [66]:
df['weather'] = weather_level

<hr>

In [67]:
df['train_direction'].value_counts() # 删除这个特征

up    50191
Name: train_direction, dtype: int64

In [68]:
df['temperature'].describe()

count    50191.000000
mean        12.913032
std          6.989936
min         -4.000000
25%          7.000000
50%         12.000000
75%         19.000000
max         28.000000
Name: temperature, dtype: float64

In [69]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50191 entries, 0 to 50190
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                50191 non-null  int64  
 1   date                      50191 non-null  object 
 2   train_number              50191 non-null  object 
 3   train_direction           50191 non-null  object 
 4   station_name              50191 non-null  object 
 5   station_order             50191 non-null  int64  
 6   scheduled_arrival_time    50191 non-null  object 
 7   scheduled_departure_time  50191 non-null  object 
 8   stop_time                 50191 non-null  object 
 9   actual_arrival_time       50191 non-null  object 
 10  actual_departure_time     50191 non-null  object 
 11  arrival_delay             50191 non-null  int64  
 12  departure_delay           50191 non-null  int64  
 13  wind                      50191 non-null  float64
 14  weathe

<hr>
时间

In [70]:
date = df['date'].apply(lambda x: x.split('-'))
date.head()

0    [2019, 10, 08]
1    [2019, 10, 08]
2    [2019, 10, 08]
3    [2019, 10, 08]
4    [2019, 10, 08]
Name: date, dtype: object

In [71]:
# 月份
mounth = date.apply(lambda x: x[1])
mounth.head()

0    10
1    10
2    10
3    10
4    10
Name: date, dtype: object

In [72]:
# 工作日5天用0表示，休息日（周六周日）表示为1  # 不需要了x
day = df['date'].apply(lambda x: datetime.datetime.strptime(x.split(" ")[0], "%Y-%m-%d").weekday())//5
day.head()

0    0
1    0
2    0
3    0
4    0
Name: date, dtype: int64

In [73]:
# 将字符串date转为datetime格式
date = df['date'].apply(lambda x:datetime.date(*map(int, x.split('-'))))
date.head()

0    2019-10-08
1    2019-10-08
2    2019-10-08
3    2019-10-08
4    2019-10-08
Name: date, dtype: object

In [74]:
# 将datetime格式转换为pd.datetime格式
date = pd.to_datetime(date, errors='coerce')

In [75]:
dayofweek = date.dt.dayofweek
dayofyear = date.dt.dayofyear

In [76]:
# 时间段（上午、下午、晚上、凌晨）的信息提取。
hour_section = df['scheduled_arrival_time'].apply(lambda x: x.split(':')[0]).astype(int)//6

In [77]:
df['mounth']=mounth
df['dayofweek']=dayofweek
df['hour_section']=hour_section

<hr>
车站1 -- one hot

车站处理二选一！！！！

In [31]:
from sklearn.preprocessing import OneHotEncoder

#进行one-hot编码
tempdata = df[['station_name']]
print(tempdata)
enc = OneHotEncoder()
enc.fit(tempdata)

#one-hot编码的结果是比较奇怪的，最好是先转换成二维数组
tempdata = enc.transform(tempdata).toarray()
print(tempdata)

#再将二维数组转换为DataFrame，记得这里会变成多列
tempdata = pd.DataFrame(tempdata)

                           station_name
0      Shanghaihongqiao Railway Station
1              Wuxidong Railway Station
2          Changzhoubei Railway Station
3            Nanjingnan Railway Station
4               Chuzhou Railway Station
...                                 ...
50186         Bangbunan Railway Station
50187        Xuzhoudong Railway Station
50188           Jinanxi Railway Station
50189        Dezhoudong Railway Station
50190         Tianjinxi Railway Station

[50191 rows x 1 columns]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [32]:
tempdata.head()

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
df=pd.concat([df,tempdata],axis=1)
df.shape

(50191, 44)

In [34]:
df.head()

,Unnamed: 0,date,train_number,train_direction,station_name,station_order,scheduled_arrival_time,scheduled_departure_time,stop_time,actual_arrival_time,...,13,14,15,16,17,18,19,20,21,22
0,0,2019-10-08,G102,up,Shanghaihongqiao Railway Station,1,06:26:00,06:26:00,----,06:25:00,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2019-10-08,G102,up,Wuxidong Railway Station,2,06:54:00,06:56:00,2,06:54:00,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,2,2019-10-08,G102,up,Changzhoubei Railway Station,3,07:13:00,07:15:00,2,07:12:00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,2019-10-08,G102,up,Nanjingnan Railway Station,4,07:46:00,07:48:00,2,07:45:00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,2019-10-08,G102,up,Chuzhou Railway Station,5,08:06:00,08:08:00,2,08:05:00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


车站2 -- 数字

In [78]:
station_num = change_object_cols(df['station_name'])
station_num

array([13, 19,  3, ...,  8,  6, 18], dtype=int64)

In [79]:
df['station_num']=station_num

<hr>
保存CSV

In [83]:
df.columns

Index(['Unnamed: 0', 'date', 'train_number', 'train_direction', 'station_name',
       'station_order', 'scheduled_arrival_time', 'scheduled_departure_time',
       'stop_time', 'actual_arrival_time', 'actual_departure_time',
       'arrival_delay', 'departure_delay', 'wind', 'weather', 'temperature',
       'major_holiday', 'wind direction', 'mounth', 'dayofweek',
       'hour_section', 'station_num'],
      dtype='object')

In [84]:
df = df.drop(columns=['Unnamed: 0', 'date', 'train_direction','station_name','scheduled_arrival_time', 'scheduled_departure_time',
       'stop_time', 'actual_arrival_time', 'actual_departure_time','departure_delay', 'wind direction'])
df.shape

(50190, 11)

In [85]:
df = df.drop(columns=['train_number'])

In [86]:
df.to_csv('temp.csv') # best

<hr>
归一化

In [22]:
df = pd.read_csv('temp.csv')
df = df.drop(columns=['Unnamed: 0'])
df.head()

,station_order,arrival_delay,wind,weather,temperature,major_holiday,mounth,dayofweek,hour_section,station_num
0,1,0,0.25,0.090909,21,False,10,1,1,13
1,2,0,0.00,0.090909,20,False,10,1,1,19
2,3,-1,0.25,0.181818,20,False,10,1,1,3
3,4,-1,0.00,0.181818,21,False,10,1,1,11
4,5,-1,0.00,0.090909,21,False,10,1,1,4


In [41]:
scaler = StandardScaler()
scaler.fit(df)
# scaler.fit(X)
std_X = scaler.transform(df)
# std_X = scaler.transform(X)
df=pd.DataFrame(std_X,columns=df.columns)
# X=pd.DataFrame(std_X,columns=X.columns)
df.head()
# X.head()

,station_order,arrival_delay,wind,weather,temperature,major_holiday,mounth,dayofweek,hour_section,station_num
0,-1.377611,-0.090008,0.926186,-0.199350,1.156941,-0.270117,0.301262,-1.009406,-1.420809,0.342703
1,-1.133024,-0.090008,-0.625599,-0.199350,1.013875,-0.270117,0.301262,-1.009406,-1.420809,1.248522
2,-0.888437,-0.105192,0.926186,0.393631,1.013875,-0.270117,0.301262,-1.009406,-1.420809,-1.166994
3,-0.643850,-0.105192,-0.625599,0.393631,1.156941,-0.270117,0.301262,-1.009406,-1.420809,0.040764
4,-0.399263,-0.105192,-0.625599,-0.199350,1.156941,-0.270117,0.301262,-1.009406,-1.420809,-1.016024


In [42]:
Y=df['arrival_delay']
X=df.drop(columns=['arrival_delay'])

<hr>

In [43]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split( X, Y, test_size=0.33, random_state=2022)
print(len(X_train),len(X_test))

33627 16563


In [44]:
y_train.shape

(33627,)

In [45]:
X_train.shape

(33627, 9)

In [46]:
y_train.describe()

count    33627.000000
mean         0.005288
std          1.016802
min         -8.532144
25%         -0.105192
50%         -0.090008
75%         -0.090008
max         12.254338
Name: arrival_delay, dtype: float64

In [47]:
X_train.head()

,station_order,wind,weather,temperature,major_holiday,mounth,dayofweek,hour_section,station_num
38516,1.312846,-0.625599,-0.792331,-0.702920,3.702099,-1.774185,-0.512183,-0.029278,-1.166994
6475,-1.377611,0.926186,-0.199350,1.013875,-0.270117,0.762472,-1.506629,-1.420809,0.342703
46145,3.024955,-0.625599,-0.199350,0.584676,-0.270117,0.531867,-0.512183,-0.029278,-0.110206
25468,-1.133024,0.926186,-0.199350,0.012411,-0.270117,0.762472,1.476710,-0.029278,-0.261176
42662,-0.643850,-0.625599,-0.199350,1.443074,-0.270117,0.301262,-0.014959,1.362253,1.399492
